# Сентимент-анализ отзывов на товары (простая версия)  

Соревновения Kaggle https://www.kaggle.com/c/product-reviews-sentiment-analysis-light  
В этом соревновании вам предстоит прогнозировать по тексту отзыва его тональность: 1 - позитивная, 0 - негативная.   

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords

### 1. Работа с тренировочными данными  

Загрузим данные и посмотрим на них

In [2]:
train = pd.read_csv(r'D:\Python\Kaggle\Sent_an_simple\products_sentiment_train.tsv', delimiter='\t', names=["text", "label"])

In [3]:
train.head(5)

,text,label
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [4]:
train.shape

(2000, 2)

Для дальнейшей работы подготовим 2 списка. Первый из них список текстов в порядке их следования в датасете:

In [5]:
t = train['text'].tolist()
t[:10]

['2 . take around 10,000 640x480 pictures .',
 'i downloaded a trial version of computer associates ez firewall and antivirus and fell in love with a computer security system all over again .',
 'the wrt54g plus the hga7t is a perfect solution if you need wireless coverage in a wider area or for a hard-walled house as was my case .',
 'i dont especially like how music files are unstructured ; basically they are just dumped into one folder with no organization , like you might have in windows explorer folders and subfolders .',
 'i was using the cheapie pail ... and it worked ok until the opening device fell apart .',
 'you can manage your profile , change the contrast of backlight , make different type of display , either list or tabbed . ',
 '11 ) minimal pause between songs - less than that on other devices - good achievement for a hard-drive based player ',
 'from the internet , woodworking books , local stores and personal opinions the hitachi m12v 3-1 / 4 hp router time and time a

Второй список меток классов соответствующих сообщениям из списка 1

In [6]:
label = train['label'].tolist()
label[:10]

[1, 1, 1, 0, 1, 1, 1, 1, 1, 1]

**Модель "мешка слов"**  
Мы импортируем класс CountVectorizer, создам экземпляр класса и обучаем модель на наших данных. Т.е. используя CountVectorizer со стандартными настройками, получаем из списка текстов матрицу признаков X.

In [7]:
vect = CountVectorizer().fit(t)
X_train = vect.transform(t)

Для прогнозирования тональности будем использовать **LogisticRegression** и проверим точности предсказания с помощью перекрестной проверки. 

In [8]:
accuracy = cross_val_score(LogisticRegression(random_state = 2), X_train, label, cv = 5)
print("Средняя точность перекр проверки: {:.2f}".format(np.mean(accuracy)))

Средняя точность перекр проверки: 0.77


Попробуем улучшить точность модели, для этого предпримем следующие действия:

**1) Минимальное число документов, в котором встречается слово**   
Зададим минимальное число документов, в котором должно встретиться указанное слово 5, так как это слово встречается только в одном, то вряди ли оно окажется в тестовых данных

In [9]:
vect = CountVectorizer(min_df=5).fit(t) 
X_train = vect.transform(t)
accuracy = cross_val_score(LogisticRegression(), X_train, label, cv = 5)
print("Средняя точность перекр проверки: {:.2f}".format(np.mean(accuracy)))

Средняя точность перекр проверки: 0.76


к сожалению точность немного снизилась
попробуем задать min_df равное 10 и 50


In [10]:
vect = CountVectorizer(min_df=10).fit(t) 
X_train = vect.transform(t) 
accuracy = cross_val_score(LogisticRegression(), X_train, label, cv = 5)
print("Средняя точность перекр проверки: {:.2f}".format(np.mean(accuracy)))

Средняя точность перекр проверки: 0.76


In [11]:
vect = CountVectorizer(min_df=50).fit(t) 
X_train = vect.transform(t) 
accuracy = cross_val_score(LogisticRegression(), X_train, label, cv = 5)
print("Средняя точность перекр проверки: {:.2f}".format(np.mean(accuracy)))

Средняя точность перекр проверки: 0.72


Видим что при min_df=10 и 5, точность имеет одинаковое значение, а при 50 снижается, поэтому для дальнейшей работы примем min_df = 5

**2) Использование стоп слов**  
Стоп слова из библиотеки sklearn

In [12]:
vect = CountVectorizer(min_df=5, stop_words="english").fit(t) 
X_train = vect.transform(t)
accuracy = cross_val_score(LogisticRegression(), X_train, label, cv = 5)
print("Средняя точность перекр проверки: {:.2f}".format(np.mean(accuracy)))

Средняя точность перекр проверки: 0.74


Стоп слова из NLTK

In [13]:
stopwords = stopwords.words('english')
print(stopwords[:10])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [14]:
vect_nltk_sw = CountVectorizer(stop_words = stopwords)
X_train_nltk_sw = vect_nltk_sw.fit_transform(t)
accuracy_nltk_sw = cross_val_score(LogisticRegression(), X_train_nltk_sw, label, cv=5)
print("Средняя правильность перекр проверки: {:.2f}".format(np.mean(accuracy_nltk_sw)))

Средняя правильность перекр проверки: 0.75


В обоих случаях точности имеют более низкое значение, чем при использовании CountVectorizer с параметрами по умолчанию

**3) N-граммы**  
В некоторых случаях это может сыграть ключевую роль - например в случае с частчкой "не", тогда отзыв может имет противоположное значение

In [15]:
cv = CountVectorizer(ngram_range=(1,2))
X_train_bi = cv.fit_transform(t)
accuracy_bi = cross_val_score(LogisticRegression(), X_train_bi, label, cv=5)
print("Средняя правильность перекр проверки: {:.2f}".format(np.mean(accuracy_bi)))

Средняя правильность перекр проверки: 0.77


In [16]:
cv3 = CountVectorizer(ngram_range=(1,3))
X_train_3 = cv.fit_transform(t)
accuracy_3 = cross_val_score(LogisticRegression(), X_train_3, label, cv=5)
print("Средняя правильность перекр проверки: {:.2f}".format(np.mean(accuracy_3)))

Средняя правильность перекр проверки: 0.77


Точность осталась на том же уровне, для дальнейшей работы будем использовать биграммы

### 2. Работа с тестовыми данными 

Загружаем данные и готовим их для дальнейшей работы

In [17]:
test = pd.read_csv(r'D:\Python\Kaggle\Sent_an_simple\products_sentiment_test.tsv', delimiter='\t')
test.head(5)

,Id,text
0,0,"so , why the small digital elph , rather than ..."
1,1,3/4 way through the first disk we played on it...
2,2,better for the zen micro is outlook compatibil...
3,3,6 . play gameboy color games on it with goboy .
4,4,"likewise , i 've heard norton 2004 professiona..."


In [18]:
test.shape

(500, 2)

In [19]:
test_data = test['text'].tolist()
test_data[:10]

["so , why the small digital elph , rather than one of the other cameras with better resolution or picture quality ? size [ + 2 ] # # because , unless it 's small , i won 't cary it around .",
 '3/4 way through the first disk we played on it ( naturally on 31 days after purchase ) the dvd player froze . ',
 'better for the zen micro is outlook compatibility .',
 '6 . play gameboy color games on it with goboy .',
 "likewise , i 've heard norton 2004 professional version is fine too .",
 'mine was 2 weeks old and i chucked it in the trash , where it belongs . ',
 "i find it very stable and comfortable to use , it doesn 't jump around or make me feel like i need to use a lot of arm strength to keep it under control , and it has very good visibility to see your cut .",
 "styling / ergonomics : the keys are small , which generally doesn 't bother me if they are spaced because i can employ a little ' braille , ' but the keys run into each other , so careful attn. must be paid when dialing ."

In [28]:
vect = CountVectorizer(min_df=5, ngram_range=(1,2)).fit(t) 
X_train = vect.transform(t)
X_test = vect.transform(test_data)

Обучим логистическую регрессию на тренировочных данных

In [29]:
clf = LogisticRegression(random_state = 2)
clf.fit(X_train, label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Сделаем предсказание

In [30]:
pred = clf.predict(X_test)

Взгляем на пример предсказания

Cохранним данные csv файл

In [31]:
df  = pd.DataFrame(pred)
df.index.name = 'Id'
df.columns = ['y']
df.head(5)

,y
Id,
0,1
1,0
2,1
3,1
4,0


In [32]:
df.to_csv(r'D:\Python\Kaggle\Sent_an_simple\submission_nn.csv', sep=',')

Результат выгрузки решения на Kaggle точность 0.78.   
Попробуем улучшить результат

### 3. Стемминг

Загружаем необходимые библиотеки и обучаем модель

In [33]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

In [34]:
stemmer = PorterStemmer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

stem_vectorizer = CountVectorizer(analyzer=stemmed_words)
vec_stem = stem_vectorizer.fit(t)
X_train_stem = vec_stem.transform(t)
X_train_stem

<2000x3013 sparse matrix of type '<class 'numpy.int64'>'
	with 29471 stored elements in Compressed Sparse Row format>

In [35]:
accuracy = cross_val_score(LogisticRegression(random_state = 2), X_train_stem, label, cv = 5)
print("Средняя точность перекр проверки: {:.2f}".format(np.mean(accuracy)))

Средняя точность перекр проверки: 0.78


Точность немного повысилась

In [36]:
stemmer = PorterStemmer()
analyzer = CountVectorizer(ngram_range=(1,2), min_df=5).build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))


stem_vectorizer2 = CountVectorizer(analyzer=stemmed_words)
vec_stem2 = stem_vectorizer2.fit(t)
X_train_stem2 = vec_stem2.transform(t)
X_train_stem2

<2000x21406 sparse matrix of type '<class 'numpy.int64'>'
	with 59720 stored elements in Compressed Sparse Row format>

In [37]:
accuracy2 = cross_val_score(LogisticRegression(random_state = 2), X_train_stem2, label, cv = 5)
print("Средняя точность перекр проверки: {:.2f}".format(np.mean(accuracy2)))

Средняя точность перекр проверки: 0.78


Сделаем предсказание на тестовых данных, запишем в файл и отправим на Kaggle

In [38]:
X_test3 = vec_stem2.transform(test_data)
clf = LogisticRegression(random_state = 2)
clf.fit(X_train_stem2, label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [39]:
pred = clf.predict(X_test3)

In [40]:
df3  = pd.DataFrame(pred)
df3.index.name = 'Id'
df3.columns = ['y']
df3.head(5)

,y
Id,
0,1
1,0
2,1
3,1
4,0


In [41]:
df3.to_csv(r'D:\Python\Kaggle\Sent_an_simple\submission_st.csv', sep=',')

Результаты на Kaggle улучшились: точность 0.81